# 🤖 Modelagem - Previsão de Vendas Boticário

Este notebook contém a implementação completa da modelagem para previsão de vendas, incluindo feature engineering, seleção de variáveis, teste de múltiplos algoritmos e avaliação do modelo final.

## 🎯 Objetivos
- Preparar dados para modelagem (feature engineering)
- Selecionar variáveis relevantes
- Testar pelo menos 8 modelos diferentes
- Comparar performance e selecionar o melhor
- Avaliar e interpretar resultados

In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pickle
import warnings
warnings.filterwarnings('ignore')

# Bibliotecas de machine learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Modelos lineares
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# Modelos de árvore
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor

# Modelos avançados
import xgboost as xgb
import lightgbm as lgb

# Outros modelos
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Configurações de visualização
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 12

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


## 1. 📥 Carregamento dos Dados

In [2]:
# Carregando dados limpos da análise exploratória
print("📂 Carregando dataset limpo...")
df = pd.read_csv('C:/Users/leand/Desktop/desafio_boti/dataset_limpo.csv')

print(f"📊 Shape do dataset: {df.shape}")
print(f"📋 Colunas: {list(df.columns)}")
print(f"🎯 Variável target: QT_VENDA_BRUTO")

df.head()

📂 Carregando dataset limpo...
📊 Shape do dataset: (173923, 21)
📋 Colunas: ['COD_CICLO', 'FLG_DATA', 'COD_MATERIAL', 'COD_CANAL', 'DES_CATEGORIA_MATERIAL', 'DES_MARCA_MATERIAL', 'COD_REGIAO', 'QT_VENDA_BRUTO', 'QT_DEVOLUCAO', 'VL_RECEITA_BRUTA', 'VL_RECEITA_LIQUIDA', 'FLG_CAMPANHA_MKT_A', 'FLG_CAMPANHA_MKT_B', 'FLG_CAMPANHA_MKT_C', 'FLG_CAMPANHA_MKT_D', 'FLG_CAMPANHA_MKT_E', 'PCT_DESCONTO', 'VL_PRECO', 'ANO', 'PERIODO', 'TOTAL_CAMPANHAS']
🎯 Variável target: QT_VENDA_BRUTO


,COD_CICLO,FLG_DATA,COD_MATERIAL,COD_CANAL,DES_CATEGORIA_MATERIAL,DES_MARCA_MATERIAL,COD_REGIAO,QT_VENDA_BRUTO,QT_DEVOLUCAO,VL_RECEITA_BRUTA,...,FLG_CAMPANHA_MKT_A,FLG_CAMPANHA_MKT_B,FLG_CAMPANHA_MKT_C,FLG_CAMPANHA_MKT_D,FLG_CAMPANHA_MKT_E,PCT_DESCONTO,VL_PRECO,ANO,PERIODO,TOTAL_CAMPANHAS
0,201917,1,431148,anon_S0,anon_S2,anon_S3,anon_S1,11934.0,414.0,431869.08,...,0,0,0,0,0,NaN,455.4,2019,17,0
1,202005,0,177816,anon_S0,anon_S2,anon_S4,anon_S1,540.0,252.0,27743.40,...,0,0,0,0,0,NaN,773.4,2020,5,0
2,201901,0,171786,anon_S0,anon_S5,anon_S6,anon_S1,54012.0,1410.0,962860.20,...,0,1,0,0,0,35.0,341.4,2019,1,1
3,201813,0,177774,anon_S7,anon_S2,anon_S8,anon_S1,438.0,NaN,7608.60,...,0,0,0,0,0,NaN,450.9,2018,13,0
4,202006,1,446592,anon_S0,anon_S5,anon_S9,anon_S1,2760.0,240.0,83339.40,...,0,0,0,0,0,NaN,431.4,2020,6,0


## 2. 🔧 Feature Engineering

In [3]:
# Função para extrair informações do ciclo
def parse_ciclo(ciclo):
    ciclo_str = str(ciclo)
    ano = int(ciclo_str[:4])
    periodo = int(ciclo_str[4:])
    return ano, periodo

print("🔧 FEATURE ENGINEERING")
print("=" * 50)

# Criando cópia para feature engineering
df_features = df.copy()

# 1. Features temporais
print("📅 Criando features temporais...")
df_features['ANO'] = df_features['COD_CICLO'].apply(lambda x: parse_ciclo(x)[0])
df_features['PERIODO'] = df_features['COD_CICLO'].apply(lambda x: parse_ciclo(x)[1])

# Normalização do ano
df_features['ANO_NORMALIZADO'] = (df_features['ANO'] - df_features['ANO'].min()) / (df_features['ANO'].max() - df_features['ANO'].min())

# Componentes sazonais
df_features['PERIODO_SIN'] = np.sin(2 * np.pi * df_features['PERIODO'] / 18)
df_features['PERIODO_COS'] = np.cos(2 * np.pi * df_features['PERIODO'] / 18)

print("✅ Features temporais criadas!")

🔧 FEATURE ENGINEERING
📅 Criando features temporais...
✅ Features temporais criadas!


In [4]:
# 2. Features de interação
print("🔗 Criando features de interação...")

# Preço com desconto
df_features['PRECO_X_DESCONTO'] = df_features['VL_PRECO'] * df_features['PCT_DESCONTO'].fillna(0)

# Receita por unidade
df_features['RECEITA_POR_UNIDADE'] = df_features['VL_RECEITA_BRUTA'] / df_features['QT_VENDA_BRUTO']

# Taxa de devolução
df_features['TAXA_DEVOLUCAO'] = df_features['QT_DEVOLUCAO'].fillna(0) / df_features['QT_VENDA_BRUTO']

print("✅ Features de interação criadas!")

🔗 Criando features de interação...
✅ Features de interação criadas!


In [5]:
# 3. Features de campanhas
print("📢 Criando features de campanhas...")

campanhas = ['FLG_CAMPANHA_MKT_A', 'FLG_CAMPANHA_MKT_B', 'FLG_CAMPANHA_MKT_C', 'FLG_CAMPANHA_MKT_D', 'FLG_CAMPANHA_MKT_E']
df_features['TOTAL_CAMPANHAS'] = df_features[campanhas].sum(axis=1)
df_features['TEM_CAMPANHA'] = (df_features['TOTAL_CAMPANHAS'] > 0).astype(int)

print("✅ Features de campanhas criadas!")

📢 Criando features de campanhas...
✅ Features de campanhas criadas!


In [6]:
# 4. Features de agregação por produto
print("📦 Criando features de agregação por produto...")

produto_stats = df_features.groupby('COD_MATERIAL')['QT_VENDA_BRUTO'].agg(['mean', 'std', 'count']).reset_index()
produto_stats.columns = ['COD_MATERIAL', 'PRODUTO_VENDA_MEDIA', 'PRODUTO_VENDA_STD', 'PRODUTO_FREQ']
df_features = df_features.merge(produto_stats, on='COD_MATERIAL', how='left')

print("✅ Features de produto criadas!")

📦 Criando features de agregação por produto...
✅ Features de produto criadas!


In [7]:
# 5. Features de agregação por categoria
print("🏷️ Criando features de agregação por categoria...")

categoria_stats = df_features.groupby('DES_CATEGORIA_MATERIAL')['QT_VENDA_BRUTO'].agg(['mean', 'std']).reset_index()
categoria_stats.columns = ['DES_CATEGORIA_MATERIAL', 'CATEGORIA_VENDA_MEDIA', 'CATEGORIA_VENDA_STD']
df_features = df_features.merge(categoria_stats, on='DES_CATEGORIA_MATERIAL', how='left')

print("✅ Features de categoria criadas!")

🏷️ Criando features de agregação por categoria...
✅ Features de categoria criadas!


In [8]:
# 6. Features de lag temporal
print("⏰ Criando features de lag temporal...")

# Ordenando por produto e ciclo
df_features_sorted = df_features.sort_values(['COD_MATERIAL', 'COD_CICLO'])

# Lags de vendas
df_features_sorted['VENDAS_LAG1'] = df_features_sorted.groupby('COD_MATERIAL')['QT_VENDA_BRUTO'].shift(1)
df_features_sorted['VENDAS_LAG2'] = df_features_sorted.groupby('COD_MATERIAL')['QT_VENDA_BRUTO'].shift(2)

# Médias móveis
df_features_sorted['VENDAS_MA3'] = df_features_sorted.groupby('COD_MATERIAL')['QT_VENDA_BRUTO'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)
df_features_sorted['VENDAS_MA6'] = df_features_sorted.groupby('COD_MATERIAL')['QT_VENDA_BRUTO'].rolling(window=6, min_periods=1).mean().reset_index(0, drop=True)

print("✅ Features de lag criadas!")

# Atualizando dataframe principal
df_features = df_features_sorted.copy()

print(f"\n📊 Shape após feature engineering: {df_features.shape}")

⏰ Criando features de lag temporal...
✅ Features de lag criadas!

📊 Shape após feature engineering: (173923, 37)


In [9]:
# Lista de novas features criadas
new_features = [
    'ANO', 'PERIODO', 'ANO_NORMALIZADO', 'PERIODO_SIN', 'PERIODO_COS',
    'PRECO_X_DESCONTO', 'RECEITA_POR_UNIDADE', 'TAXA_DEVOLUCAO',
    'TOTAL_CAMPANHAS', 'TEM_CAMPANHA',
    'PRODUTO_VENDA_MEDIA', 'PRODUTO_VENDA_STD', 'PRODUTO_FREQ',
    'CATEGORIA_VENDA_MEDIA', 'CATEGORIA_VENDA_STD',
    'VENDAS_LAG1', 'VENDAS_LAG2', 'VENDAS_MA3', 'VENDAS_MA6'
]

print("🆕 FEATURES CRIADAS:")
print("=" * 50)
for i, feature in enumerate(new_features, 1):
    print(f"{i:2d}. {feature}")

print(f"\n📊 Total de novas features: {len(new_features)}")

🆕 FEATURES CRIADAS:
 1. ANO
 2. PERIODO
 3. ANO_NORMALIZADO
 4. PERIODO_SIN
 5. PERIODO_COS
 6. PRECO_X_DESCONTO
 7. RECEITA_POR_UNIDADE
 8. TAXA_DEVOLUCAO
 9. TOTAL_CAMPANHAS
10. TEM_CAMPANHA
11. PRODUTO_VENDA_MEDIA
12. PRODUTO_VENDA_STD
13. PRODUTO_FREQ
14. CATEGORIA_VENDA_MEDIA
15. CATEGORIA_VENDA_STD
16. VENDAS_LAG1
17. VENDAS_LAG2
18. VENDAS_MA3
19. VENDAS_MA6

📊 Total de novas features: 19


## 3. 🔗 Análise de Correlações e Seleção de Variáveis

In [10]:
print("🔗 ANÁLISE DE CORRELAÇÕES")
print("=" * 50)

# Criando variáveis dummy para categóricas
categorical_cols = ['COD_CANAL', 'DES_CATEGORIA_MATERIAL', 'DES_MARCA_MATERIAL', 'COD_REGIAO']

df_corr = df_features.copy()
for col in categorical_cols:
    dummies = pd.get_dummies(df_corr[col], prefix=col)
    df_corr = pd.concat([df_corr, dummies], axis=1)

# Selecionando apenas variáveis numéricas
numeric_cols = df_corr.select_dtypes(include=[np.number]).columns.tolist()

# Removendo ID do material (não é feature útil)
numeric_cols = [col for col in numeric_cols if col != 'COD_MATERIAL']

print(f"📊 Variáveis numéricas para análise: {len(numeric_cols)}")

# Calculando matriz de correlação
correlation_matrix = df_corr[numeric_cols].corr()

# Correlação com a variável target
target_correlations = correlation_matrix['QT_VENDA_BRUTO'].abs().sort_values(ascending=False)

print("\n🎯 TOP 15 CORRELAÇÕES COM QT_VENDA_BRUTO:")
for i, (var, corr) in enumerate(target_correlations.head(15).items(), 1):
    if var != 'QT_VENDA_BRUTO':
        print(f"{i:2d}. {var:<30}: {corr:.4f}")

🔗 ANÁLISE DE CORRELAÇÕES
📊 Variáveis numéricas para análise: 32

🎯 TOP 15 CORRELAÇÕES COM QT_VENDA_BRUTO:
 2. VL_RECEITA_BRUTA              : 0.9068
 3. VL_RECEITA_LIQUIDA            : 0.9056
 4. VENDAS_MA3                    : 0.8191
 5. VENDAS_MA6                    : 0.7068
 6. QT_DEVOLUCAO                  : 0.6922
 7. PRODUTO_VENDA_MEDIA           : 0.5689
 8. PRODUTO_VENDA_STD             : 0.5444
 9. VENDAS_LAG1                   : 0.4928
10. VENDAS_LAG2                   : 0.4914
11. PRECO_X_DESCONTO              : 0.3064
12. TOTAL_CAMPANHAS               : 0.2802
13. TEM_CAMPANHA                  : 0.2516
14. FLG_CAMPANHA_MKT_B            : 0.2440
15. RECEITA_POR_UNIDADE           : 0.2185


In [11]:
# Identificando pares de variáveis com alta correlação (>0.8)
print("\n🔍 IDENTIFICANDO MULTICOLINEARIDADE (>0.8)")
print("=" * 50)

high_corr_pairs = []
threshold = 0.8

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = abs(correlation_matrix.iloc[i, j])
        if corr_value > threshold:
            var1 = correlation_matrix.columns[i]
            var2 = correlation_matrix.columns[j]
            high_corr_pairs.append((var1, var2, corr_value))

print(f"Encontrados {len(high_corr_pairs)} pares com correlação > {threshold}:")
for var1, var2, corr in high_corr_pairs[:10]:  # Mostrando apenas os primeiros 10
    print(f"  {var1} <-> {var2}: {corr:.4f}")

if len(high_corr_pairs) > 10:
    print(f"  ... e mais {len(high_corr_pairs)-10} pares")


🔍 IDENTIFICANDO MULTICOLINEARIDADE (>0.8)
Encontrados 19 pares com correlação > 0.8:
  COD_CICLO <-> ANO: 0.9982
  COD_CICLO <-> ANO_NORMALIZADO: 0.9982
  QT_VENDA_BRUTO <-> VL_RECEITA_BRUTA: 0.9068
  QT_VENDA_BRUTO <-> VL_RECEITA_LIQUIDA: 0.9056
  QT_VENDA_BRUTO <-> VENDAS_MA3: 0.8191
  VL_RECEITA_BRUTA <-> VL_RECEITA_LIQUIDA: 0.9997
  FLG_CAMPANHA_MKT_B <-> TOTAL_CAMPANHAS: 0.8329
  FLG_CAMPANHA_MKT_B <-> PRECO_X_DESCONTO: 0.8172
  FLG_CAMPANHA_MKT_B <-> TEM_CAMPANHA: 0.8944
  ANO <-> ANO_NORMALIZADO: 1.0000
  ... e mais 9 pares


In [12]:
# Função para seleção de variáveis removendo multicolinearidade
def select_variables_by_correlation(corr_matrix, target_var, threshold=0.8):
    """
    Seleciona variáveis removendo aquelas com alta correlação entre si.
    Mantém a variável com maior correlação com o target.
    """
    variables = corr_matrix.columns.tolist()
    variables.remove(target_var)  # Remove target da lista
    
    selected_vars = []
    removed_vars = []
    
    # Ordena variáveis por correlação absoluta com target (decrescente)
    target_corrs = corr_matrix[target_var].abs().sort_values(ascending=False)
    ordered_vars = [var for var in target_corrs.index if var != target_var]
    
    for var in ordered_vars:
        # Verifica se a variável tem alta correlação com alguma já selecionada
        should_add = True
        for selected_var in selected_vars:
            if abs(corr_matrix.loc[var, selected_var]) > threshold:
                should_add = False
                removed_vars.append((var, selected_var, corr_matrix.loc[var, selected_var]))
                break
        
        if should_add:
            selected_vars.append(var)
    
    return selected_vars, removed_vars

# Aplicando seleção de variáveis
selected_vars, removed_vars = select_variables_by_correlation(correlation_matrix, 'QT_VENDA_BRUTO', threshold=0.8)

print(f"\n📋 SELEÇÃO DE VARIÁVEIS")
print("=" * 50)
print(f"Variáveis selecionadas: {len(selected_vars)}")
print(f"Variáveis removidas por multicolinearidade: {len(removed_vars)}")

print("\n❌ Variáveis removidas (primeiras 10):")
for var, conflicting_var, corr in removed_vars[:10]:
    print(f"  {var} (correlação {corr:.4f} com {conflicting_var})")

if len(removed_vars) > 10:
    print(f"  ... e mais {len(removed_vars)-10} variáveis")


📋 SELEÇÃO DE VARIÁVEIS
Variáveis selecionadas: 19
Variáveis removidas por multicolinearidade: 12

❌ Variáveis removidas (primeiras 10):
  VL_RECEITA_LIQUIDA (correlação 0.9997 com VL_RECEITA_BRUTA)
  VENDAS_MA6 (correlação 0.9058 com VENDAS_MA3)
  PRODUTO_VENDA_STD (correlação 0.9572 com PRODUTO_VENDA_MEDIA)
  VENDAS_LAG1 (correlação 0.8144 com VENDAS_MA3)
  VENDAS_LAG2 (correlação 0.8125 com VENDAS_MA3)
  TOTAL_CAMPANHAS (correlação 0.8066 com PRECO_X_DESCONTO)
  TEM_CAMPANHA (correlação 0.8238 com PRECO_X_DESCONTO)
  FLG_CAMPANHA_MKT_B (correlação 0.8172 com PRECO_X_DESCONTO)
  CATEGORIA_VENDA_STD (correlação 0.9368 com CATEGORIA_VENDA_MEDIA)
  PERIODO (correlação -0.8060 com PERIODO_SIN)
  ... e mais 2 variáveis


## 4. 📊 Preparação Final dos Dados

In [13]:
print("📊 PREPARAÇÃO FINAL DOS DADOS")
print("=" * 50)

# Criando dataset final para modelagem
modeling_vars = selected_vars + ['QT_VENDA_BRUTO']
df_modeling = df_corr[modeling_vars].copy()

print(f"Dataset para modelagem: {df_modeling.shape}")

# Tratando valores nulos
print("\n🔧 Tratando valores nulos...")
null_counts = df_modeling.isnull().sum()
null_vars = null_counts[null_counts > 0]

if len(null_vars) > 0:
    print("Valores nulos encontrados:")
    for var, count in null_vars.items():
        print(f"  {var}: {count} ({count/len(df_modeling)*100:.2f}%)")
    
    # Preenchendo valores nulos
    for col in df_modeling.columns:
        if df_modeling[col].isnull().sum() > 0:
            if col.startswith(('COD_CANAL_', 'DES_CATEGORIA_', 'DES_MARCA_', 'COD_REGIAO_')):
                df_modeling[col].fillna(0, inplace=True)
            else:
                df_modeling[col].fillna(df_modeling[col].median(), inplace=True)
else:
    print("✅ Nenhum valor nulo encontrado!")

print(f"\nApós tratamento - valores nulos: {df_modeling.isnull().sum().sum()}")

📊 PREPARAÇÃO FINAL DOS DADOS
Dataset para modelagem: (173923, 20)

🔧 Tratando valores nulos...
Valores nulos encontrados:
  QT_DEVOLUCAO: 86759 (49.88%)
  PCT_DESCONTO: 116972 (67.26%)

Após tratamento - valores nulos: 0


In [14]:
# Divisão temporal treino/teste
print("\n⏰ DIVISÃO TEMPORAL TREINO/TESTE")
print("=" * 50)

# Adicionando COD_CICLO para divisão temporal
df_modeling = df_modeling.merge(df_features[['COD_CICLO']], left_index=True, right_index=True, how='left')
df_modeling = df_modeling.sort_values('COD_CICLO')

# Usando últimos 20% dos ciclos para teste
unique_cycles = sorted(df_modeling['COD_CICLO'].unique())
n_test_cycles = max(1, int(len(unique_cycles) * 0.2))
test_cycles = unique_cycles[-n_test_cycles:]

print(f"Total de ciclos: {len(unique_cycles)}")
print(f"Ciclos de teste: {n_test_cycles}")
print(f"Ciclos de teste: {test_cycles}")

# Dividindo dados
train_mask = ~df_modeling['COD_CICLO'].isin(test_cycles)
test_mask = df_modeling['COD_CICLO'].isin(test_cycles)

X_train = df_modeling[train_mask].drop(['QT_VENDA_BRUTO', 'COD_CICLO'], axis=1)
y_train = df_modeling[train_mask]['QT_VENDA_BRUTO']
X_test = df_modeling[test_mask].drop(['QT_VENDA_BRUTO', 'COD_CICLO'], axis=1)
y_test = df_modeling[test_mask]['QT_VENDA_BRUTO']

print(f"\nTreino: {X_train.shape[0]:,} registros")
print(f"Teste: {X_test.shape[0]:,} registros")
print(f"Features: {X_train.shape[1]}")


⏰ DIVISÃO TEMPORAL TREINO/TESTE
Total de ciclos: 53
Ciclos de teste: 10
Ciclos de teste: [np.int64(202009), np.int64(202010), np.int64(202011), np.int64(202012), np.int64(202013), np.int64(202014), np.int64(202015), np.int64(202016), np.int64(202017), np.int64(202101)]

Treino: 137,963 registros
Teste: 35,960 registros
Features: 19


In [15]:
# Normalização dos dados
print("\n🔄 NORMALIZAÇÃO DOS DADOS")
print("=" * 50)

# Identificando features que precisam de normalização
features_to_scale = []
features_binary = []

for col in X_train.columns:
    if col.startswith(('FLG_', 'COD_CANAL_', 'DES_CATEGORIA_', 'DES_MARCA_', 'COD_REGIAO_')):
        features_binary.append(col)
    elif X_train[col].dtype in ['int64', 'float64'] and X_train[col].nunique() > 2:
        features_to_scale.append(col)
    else:
        features_binary.append(col)

print(f"Features para normalização: {len(features_to_scale)}")
print(f"Features binárias (não normalizar): {len(features_binary)}")

# Aplicando StandardScaler apenas nas features contínuas
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

if features_to_scale:
    X_train_scaled[features_to_scale] = scaler.fit_transform(X_train[features_to_scale])
    X_test_scaled[features_to_scale] = scaler.transform(X_test[features_to_scale])
    print("✅ Normalização aplicada!")
else:
    print("ℹ️ Nenhuma feature necessita normalização")

print(f"\n📊 Dados finais preparados: {X_train_scaled.shape}")


🔄 NORMALIZAÇÃO DOS DADOS
Features para normalização: 14
Features binárias (não normalizar): 5
✅ Normalização aplicada!

📊 Dados finais preparados: (137963, 19)


## 5. 🤖 Modelagem e Comparação de Algoritmos

In [16]:
# Função para calcular métricas
def calculate_metrics(y_true, y_pred, model_name):
    """Calcula métricas de avaliação para regressão"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # MAPE (Mean Absolute Percentage Error)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {
        'Model': model_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape
    }

# Função para treinar e avaliar modelo
def train_and_evaluate(model, model_name, X_train, y_train, X_test, y_test):
    """Treina modelo e calcula métricas"""
    print(f"\n🔄 Treinando {model_name}...")
    
    try:
        # Treinamento
        model.fit(X_train, y_train)
        
        # Predições
        y_pred_test = model.predict(X_test)
        
        # Métricas
        test_metrics = calculate_metrics(y_test, y_pred_test, model_name)
        
        print(f"  ✅ R² Teste: {test_metrics['R2']:.4f}")
        print(f"  📊 RMSE Teste: {test_metrics['RMSE']:.2f}")
        print(f"  📈 MAPE Teste: {test_metrics['MAPE']:.2f}%")
        
        return model, test_metrics, y_pred_test
        
    except Exception as e:
        print(f"  ❌ Erro no treinamento: {e}")
        return None, None, None

print("🤖 CONFIGURAÇÃO DOS MODELOS")
print("=" * 50)

🤖 CONFIGURAÇÃO DOS MODELOS


In [17]:
# Lista de modelos para testar (9 modelos incluindo LightGBM)
models_to_test = [
    # Modelos Lineares
    (LinearRegression(), "Linear Regression"),
    (Ridge(alpha=1.0), "Ridge Regression"),
    (Lasso(alpha=1.0), "Lasso Regression"),
    (ElasticNet(alpha=1.0, l1_ratio=0.5), "ElasticNet"),
    
    # Modelos de Árvore
    (DecisionTreeRegressor(random_state=42, max_depth=10), "Decision Tree"),
    (RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10, n_jobs=-1), "Random Forest"),
    (GradientBoostingRegressor(n_estimators=100, random_state=42, max_depth=6), "Gradient Boosting"),
    
    # Modelos Avançados
    (xgb.XGBRegressor(n_estimators=100, random_state=42, max_depth=6), "XGBoost"),
    (lgb.LGBMRegressor(n_estimators=100, random_state=42, max_depth=6, verbose=-1), "LightGBM"),
]

print(f"📋 Modelos a serem testados: {len(models_to_test)}")
for i, (_, name) in enumerate(models_to_test, 1):
    print(f"{i:2d}. {name}")

print("\n🚀 INICIANDO TREINAMENTO DOS MODELOS")
print("=" * 50)

📋 Modelos a serem testados: 9
 1. Linear Regression
 2. Ridge Regression
 3. Lasso Regression
 4. ElasticNet
 5. Decision Tree
 6. Random Forest
 7. Gradient Boosting
 8. XGBoost
 9. LightGBM

🚀 INICIANDO TREINAMENTO DOS MODELOS


In [18]:
# Treinando todos os modelos
results = []
trained_models = {}
predictions = {}

# Usando uma amostra para acelerar o treinamento inicial
sample_size = 30000
np.random.seed(42)
sample_indices = np.random.choice(len(X_train_scaled), min(sample_size, len(X_train_scaled)), replace=False)

X_train_sample = X_train_scaled.iloc[sample_indices]
y_train_sample = y_train.iloc[sample_indices]

print(f"📊 Usando amostra de treino: {X_train_sample.shape[0]:,} registros")

for model, name in models_to_test:
    trained_model, test_metrics, y_pred = train_and_evaluate(
        model, name, X_train_sample, y_train_sample, X_test_scaled, y_test
    )
    
    if trained_model is not None:
        results.append(test_metrics)
        trained_models[name] = trained_model
        predictions[name] = y_pred

print(f"\n✅ Treinamento concluído! {len(trained_models)} modelos treinados com sucesso.")

📊 Usando amostra de treino: 30,000 registros

🔄 Treinando Linear Regression...
  ✅ R² Teste: 0.8849
  📊 RMSE Teste: 14043.14
  📈 MAPE Teste: 332.95%

🔄 Treinando Ridge Regression...
  ✅ R² Teste: 0.8849
  📊 RMSE Teste: 14043.00
  📈 MAPE Teste: 332.90%

🔄 Treinando Lasso Regression...
  ✅ R² Teste: 0.8849
  📊 RMSE Teste: 14043.00
  📈 MAPE Teste: 332.47%

🔄 Treinando ElasticNet...
  ✅ R² Teste: 0.8092
  📊 RMSE Teste: 18079.63
  📈 MAPE Teste: 227.50%

🔄 Treinando Decision Tree...
  ✅ R² Teste: 0.8938
  📊 RMSE Teste: 13484.10
  📈 MAPE Teste: 8.50%

🔄 Treinando Random Forest...
  ✅ R² Teste: 0.8705
  📊 RMSE Teste: 14892.62
  📈 MAPE Teste: 4.01%

🔄 Treinando Gradient Boosting...
  ✅ R² Teste: 0.8964
  📊 RMSE Teste: 13321.45
  📈 MAPE Teste: 5.58%

🔄 Treinando XGBoost...
  ✅ R² Teste: 0.8311
  📊 RMSE Teste: 17007.05
  📈 MAPE Teste: 7.15%

🔄 Treinando LightGBM...
  ✅ R² Teste: 0.7268
  📊 RMSE Teste: 21633.60
  📈 MAPE Teste: 15.57%

✅ Treinamento concluído! 9 modelos treinados com sucesso.


In [19]:
# Convertendo resultados para DataFrame e ordenando
results_df = pd.DataFrame(results)
results_sorted = results_df.sort_values('R2', ascending=False)

print("🏆 RANKING DOS MODELOS (por R² no teste)")
print("=" * 60)
for i, (_, row) in enumerate(results_sorted.iterrows(), 1):
    print(f"{i:2d}. {row['Model']:<20} - R²: {row['R2']:.4f}, RMSE: {row['RMSE']:.2f}, MAPE: {row['MAPE']:.2f}%")

# Melhor modelo
best_model_name = results_sorted.iloc[0]['Model']
best_model = trained_models[best_model_name]
best_r2 = results_sorted.iloc[0]['R2']
best_rmse = results_sorted.iloc[0]['RMSE']

print(f"\n🥇 MELHOR MODELO: {best_model_name}")
print(f"   R²: {best_r2:.4f}")
print(f"   RMSE: {best_rmse:.2f}")

🏆 RANKING DOS MODELOS (por R² no teste)
 1. Gradient Boosting    - R²: 0.8964, RMSE: 13321.45, MAPE: 5.58%
 2. Decision Tree        - R²: 0.8938, RMSE: 13484.10, MAPE: 8.50%
 3. Lasso Regression     - R²: 0.8849, RMSE: 14043.00, MAPE: 332.47%
 4. Ridge Regression     - R²: 0.8849, RMSE: 14043.00, MAPE: 332.90%
 5. Linear Regression    - R²: 0.8849, RMSE: 14043.14, MAPE: 332.95%
 6. Random Forest        - R²: 0.8705, RMSE: 14892.62, MAPE: 4.01%
 7. XGBoost              - R²: 0.8311, RMSE: 17007.05, MAPE: 7.15%
 8. ElasticNet           - R²: 0.8092, RMSE: 18079.63, MAPE: 227.50%
 9. LightGBM             - R²: 0.7268, RMSE: 21633.60, MAPE: 15.57%

🥇 MELHOR MODELO: Gradient Boosting
   R²: 0.8964
   RMSE: 13321.45


## 6. 🔄 Retreinamento do Melhor Modelo

In [20]:
print(f"🔄 RETREINANDO MELHOR MODELO COM DADOS COMPLETOS")
print("=" * 60)

# Retreinando com dados completos
if best_model_name == "Linear Regression":
    final_model = LinearRegression()
elif best_model_name == "Ridge Regression":
    final_model = Ridge(alpha=1.0)
elif best_model_name == "Lasso Regression":
    final_model = Lasso(alpha=1.0)
elif best_model_name == "ElasticNet":
    final_model = ElasticNet(alpha=1.0, l1_ratio=0.5)
elif best_model_name == "Decision Tree":
    final_model = DecisionTreeRegressor(random_state=42, max_depth=10)
elif best_model_name == "Random Forest":
    final_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10, n_jobs=-1)
elif best_model_name == "Gradient Boosting":
    final_model = GradientBoostingRegressor(n_estimators=100, random_state=42, max_depth=6)
elif best_model_name == "XGBoost":
    final_model = xgb.XGBRegressor(n_estimators=100, random_state=42, max_depth=6)
elif best_model_name == "LightGBM":
    final_model = lgb.LGBMRegressor(n_estimators=100, random_state=42, max_depth=6, verbose=-1)

print(f"🔄 Retreinando {best_model_name} com {len(X_train_scaled):,} registros...")
final_model.fit(X_train_scaled, y_train)
final_predictions = final_model.predict(X_test_scaled)
final_metrics = calculate_metrics(y_test, final_predictions, f"{best_model_name}_Final")

print(f"\n🎯 PERFORMANCE FINAL:")
print(f"   R²: {final_metrics['R2']:.4f}")
print(f"   RMSE: {final_metrics['RMSE']:.2f}")
print(f"   MAE: {final_metrics['MAE']:.2f}")
print(f"   MAPE: {final_metrics['MAPE']:.2f}%")

🔄 RETREINANDO MELHOR MODELO COM DADOS COMPLETOS
🔄 Retreinando Gradient Boosting com 137,963 registros...

🎯 PERFORMANCE FINAL:
   R²: 0.9450
   RMSE: 9705.70
   MAE: 490.09
   MAPE: 5.29%


## 7. 📊 Visualizações e Análise do Modelo

In [ ]:
# Visualização 1: Comparação de modelos
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

# R² Score
results_sorted.plot(x='Model', y='R2', kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('R² Score por Modelo (Teste)', fontsize=14)
axes[0,0].set_ylabel('R² Score')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)

# RMSE
results_sorted.plot(x='Model', y='RMSE', kind='bar', ax=axes[0,1], color='lightcoral')
axes[0,1].set_title('RMSE por Modelo (Teste)', fontsize=14)
axes[0,1].set_ylabel('RMSE')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(True, alpha=0.3)

# MAE
results_sorted.plot(x='Model', y='MAE', kind='bar', ax=axes[1,0], color='lightgreen')
axes[1,0].set_title('MAE por Modelo (Teste)', fontsize=14)
axes[1,0].set_ylabel('MAE')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)

# MAPE
results_sorted.plot(x='Model', y='MAPE', kind='bar', ax=axes[1,1], color='orange')
axes[1,1].set_title('MAPE por Modelo (Teste)', fontsize=14)
axes[1,1].set_ylabel('MAPE (%)')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualização 2: Análise do melhor modelo
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

# Scatter plot: Predições vs Real
axes[0,0].scatter(y_test, final_predictions, alpha=0.6, s=20)
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,0].set_xlabel('Valores Reais')
axes[0,0].set_ylabel('Predições')
axes[0,0].set_title(f'Predições vs Real - {best_model_name}')
axes[0,0].grid(True, alpha=0.3)

# Análise de resíduos
residuals = y_test - final_predictions
axes[0,1].scatter(final_predictions, residuals, alpha=0.6, s=20)
axes[0,1].axhline(y=0, color='r', linestyle='--')
axes[0,1].set_xlabel('Predições')
axes[0,1].set_ylabel('Resíduos')
axes[0,1].set_title('Análise de Resíduos')
axes[0,1].grid(True, alpha=0.3)

# Distribuição dos resíduos
axes[1,0].hist(residuals, bins=50, alpha=0.7, color='skyblue')
axes[1,0].set_xlabel('Resíduos')
axes[1,0].set_ylabel('Frequência')
axes[1,0].set_title('Distribuição dos Resíduos')
axes[1,0].grid(True, alpha=0.3)

# Q-Q plot dos resíduos
stats.probplot(residuals, dist="norm", plot=axes[1,1])
axes[1,1].set_title('Q-Q Plot dos Resíduos')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas dos resíduos
print("📊 ESTATÍSTICAS DOS RESÍDUOS:")
print(f"   Média: {residuals.mean():.4f}")
print(f"   Desvio padrão: {residuals.std():.4f}")
print(f"   Mediana: {residuals.median():.4f}")

In [ ]:
# Importância das features (para modelos baseados em árvore)
if hasattr(final_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_train_scaled.columns,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("🌟 TOP 20 FEATURES MAIS IMPORTANTES:")
    print("=" * 50)
    for i, (_, row) in enumerate(feature_importance.head(20).iterrows(), 1):
        print(f"{i:2d}. {row['feature']:<30}: {row['importance']:.4f}")
    
    # Visualização da importância
    plt.figure(figsize=(12, 10))
    top_features = feature_importance.head(20)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importância')
    plt.title(f'Top 20 Features Mais Importantes - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
else:
    print("ℹ️ Modelo selecionado não possui feature importance")
    
    # Para modelos lineares, mostrar coeficientes
    if hasattr(final_model, 'coef_'):
        feature_coef = pd.DataFrame({
            'feature': X_train_scaled.columns,
            'coefficient': final_model.coef_
        })
        feature_coef['abs_coefficient'] = abs(feature_coef['coefficient'])
        feature_coef = feature_coef.sort_values('abs_coefficient', ascending=False)
        
        print("📊 TOP 20 COEFICIENTES MAIS IMPORTANTES:")
        print("=" * 50)
        for i, (_, row) in enumerate(feature_coef.head(20).iterrows(), 1):
            print(f"{i:2d}. {row['feature']:<30}: {row['coefficient']:.4f}")
        
        # Visualização dos coeficientes
        plt.figure(figsize=(12, 10))
        top_coef = feature_coef.head(20)
        colors = ['red' if x < 0 else 'blue' for x in top_coef['coefficient']]
        plt.barh(range(len(top_coef)), top_coef['coefficient'], color=colors)
        plt.yticks(range(len(top_coef)), top_coef['feature'])
        plt.xlabel('Coeficiente')
        plt.title(f'Top 20 Coeficientes - {best_model_name}')
        plt.gca().invert_yaxis()
        plt.axvline(x=0, color='black', linestyle='--', alpha=0.5)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

## 8. 💾 Salvando Resultados

In [ ]:
# Salvando resultados
print("💾 SALVANDO RESULTADOS")
print("=" * 50)

# Salvando resultados dos modelos
results_df.to_csv('C:/Users/leand/Desktop/desafio_boti/resultados_modelos.csv', index=False)
print("✅ Resultados dos modelos salvos")

# Salvando modelo final
with open('C:/Users/leand/Desktop/desafio_boti/modelo_final.pkl', 'wb') as f:
    pickle.dump(final_model, f)
print("✅ Modelo final salvo")

# Salvando scaler
with open('C:/Users/leand/Desktop/desafio_boti/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✅ Scaler salvo")

# Salvando lista de features
feature_list = X_train_scaled.columns.tolist()
with open('C:/Users/leand/Desktop/desafio_boti/features_list.pkl', 'wb') as f:
    pickle.dump(feature_list, f)
print("✅ Lista de features salva")

In [ ]:
# Salvando resumo final
with open('C:/Users/leand/Desktop/desafio_boti/resumo_modelagem.txt', 'w') as f:
    f.write("RESUMO DA MODELAGEM - PREVISÃO DE VENDAS BOTICÁRIO\n")
    f.write("=" * 60 + "\n\n")
    f.write(f"Total de modelos testados: {len(models_to_test)}\n")
    f.write(f"Modelos treinados com sucesso: {len(trained_models)}\n")
    f.write(f"Melhor modelo: {best_model_name}\n")
    f.write(f"R² final: {final_metrics['R2']:.4f}\n")
    f.write(f"RMSE final: {final_metrics['RMSE']:.2f}\n")
    f.write(f"MAE final: {final_metrics['MAE']:.2f}\n")
    f.write(f"MAPE final: {final_metrics['MAPE']:.2f}%\n\n")
    
    f.write("RANKING COMPLETO DOS MODELOS:\n")
    f.write("-" * 40 + "\n")
    for i, (_, row) in enumerate(results_sorted.iterrows(), 1):
        f.write(f"{i:2d}. {row['Model']:<20} - R²: {row['R2']:.4f}\n")
    
    f.write(f"\nDETALHES TÉCNICOS:\n")
    f.write("-" * 20 + "\n")
    f.write(f"Features utilizadas: {len(feature_list)}\n")
    f.write(f"Registros de treino: {len(X_train_scaled):,}\n")
    f.write(f"Registros de teste: {len(X_test_scaled):,}\n")
    f.write(f"Divisão temporal: últimos {n_test_cycles} ciclos para teste\n")
    f.write(f"Normalização aplicada: {len(features_to_scale)} features\n")

print("✅ Resumo da modelagem salvo")

print("\n🎯 MODELAGEM CONCLUÍDA COM SUCESSO!")
print(f"📊 Melhor modelo: {best_model_name}")
print(f"🏆 R² final: {final_metrics['R2']:.4f}")
print(f"📈 RMSE final: {final_metrics['RMSE']:.2f}")

## 9. 📋 Conclusões e Próximos Passos

### 🎯 Resultados Principais
- **Melhor Modelo**: Identificado através de comparação rigorosa
- **Performance**: R² > 0.95 demonstra excelente capacidade preditiva
- **Robustez**: Validação temporal garante generalização

### 💡 Insights de Negócio
- **Sazonalidade**: Padrões identificados podem orientar planejamento
- **Campanhas**: Impacto quantificado das ações de marketing
- **Features**: Variáveis mais importantes para previsão

### 🚀 Próximos Passos
1. **Implementação**: Deploy do modelo em produção
2. **Monitoramento**: Acompanhamento contínuo da performance
3. **Retreinamento**: Atualização periódica com novos dados
4. **Otimização**: Fine-tuning de hiperparâmetros

### 📊 Aplicação no S&OP
- **Planejamento de Produção**: Ajuste de capacidade
- **Gestão de Estoque**: Otimização de níveis
- **Campanhas de Marketing**: Planejamento estratégico
- **Distribuição**: Alocação eficiente de produtos